## Day 7: Bridge Repair

You ask how long it'll take; the engineers tell you that it only needs final calibrations, but some young elephants were playing nearby and stole all the operators from their calibration equations! They could finish the calibrations if only someone could determine which test values could possibly be produced by placing any combination of operators into their calibration equations (your puzzle input).

For example:

In [ ]:
import sys
sys.path.append('..')
from utils import *

inp = get_data(day=7, year=2024)
samp = '''190: 10 19
3267: 81 40 27
83: 17 5
156: 15 6
7290: 6 8 6 15
161011: 16 10 13
192: 17 8 14
21037: 9 7 18 13
292: 11 6 16 20'''
print(samp)

190: 10 19
3267: 81 40 27
83: 17 5
156: 15 6
7290: 6 8 6 15
161011: 16 10 13
192: 17 8 14
21037: 9 7 18 13
292: 11 6 16 20


Each line represents a single equation. The test value appears before the colon on each line; it is your job to determine whether the remaining numbers can be combined with operators to produce the test value.

Operators are always evaluated left-to-right, not according to precedence rules. Furthermore, numbers in the equations cannot be rearranged. Glancing into the jungle, you can see elephants holding two different types of operators: add (+) and multiply (*).

Only three of the above equations can be made true by inserting operators:

- `190: 10 19` has only one position that accepts an operator: between 10 and 19. Choosing + would give 29, but choosing * would give the test value (10 * 19 = 190).
- `3267: 81 40 27` has two positions for operators. Of the four possible configurations of the operators, two cause the right side to match the test value: 81 + 40 * 27 and 81 * 40 + 27 both equal 3267 (when evaluated left-to-right)!
- `292: 11 6 16 20` can be solved in exactly one way: 11 + 6 * 16 + 20.
The engineers just need the total calibration result, which is the sum of the test values from just the equations that could possibly be true. In the above example, the sum of the test values for the three equations listed above is 3749.

Determine which equations could possibly be true. What is their total calibration result?

## Observations:
- One can use recursion. For example, for `3267: 81 40 27` one can check `81*40=3240` and the rest of the assignment is a smaller task of the same type - `3267: 3240 27`.
- Once the partial result is larger than the number on the left, I can stop (a dead end of the search).
- Solving of equations can run in parallel.

xs = samp.splitlines()

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

You've split the sample input into lines! That's a good first step for parsing the data.

What would you like to do next? Are you thinking about:
- Parsing each line to extract the test value and the numbers?
- Starting to implement the recursive solution you outlined in your observations?
- Something else?

In [ ]:
xs = samp.splitlines()
xs

['190: 10 19',
 '3267: 81 40 27',
 '83: 17 5',
 '156: 15 6',
 '7290: 6 8 6 15',
 '161011: 16 10 13',
 '192: 17 8 14',
 '21037: 9 7 18 13',
 '292: 11 6 16 20']

In [ ]:
x = xs[1]
x

'3267: 81 40 27'

In [ ]:
fst, snd = x.split(':')
fst, snd

('3267', ' 81 40 27')

In [ ]:
L(snd.split()).map(int)

(#3) [81,40,27]

In [ ]:
def parse(x):
    fst, snd = x.split(':')
    return int(fst), L(snd.split()).map(int)
eq = parse(x)
eq

(3267, (#3) [81,40,27])

In [ ]:
from operator import mul, add

def search_rec(val, nums, ops):
    if len(nums) == 1:
        return val == nums[0], ops
    # TODO: add early stop

    fst, snd, rest = nums[0], nums[1], nums[2:]
    
    # test multiplication:
    mul_res, mul_ops = search_rec(val, [mul(fst, snd)] + rest, ops + [mul])
    if mul_res:
        return mul_res, mul_ops
    
    # test addition:
    return search_rec(val, [add(fst, snd)] + rest, ops + [add])

search_rec(eq[0], eq[1], [])

(True, [<function _operator.mul(a, b, /)>, <function _operator.add(a, b, /)>])

In [ ]:
def search(val, nums): return search_rec(val, nums, [])
search(eq[0], eq[1])

(True, [<function _operator.mul(a, b, /)>, <function _operator.add(a, b, /)>])

In [ ]:
eqs = L(xs).map(parse)
eqs

(#9) [(190, [10, 19]),(3267, [81, 40, 27]),(83, [17, 5]),(156, [15, 6]),(7290, [6, 8, 6, 15]),(161011, [16, 10, 13]),(192, [17, 8, 14]),(21037, [9, 7, 18, 13]),(292, [11, 6, 16, 20])]

In [ ]:
sols = eqs.starmap(search).map(first)
sols

(#9) [True,True,False,False,False,False,False,False,True]

In [ ]:
vals = eqs.map(first)
vals

(#9) [190,3267,83,156,7290,161011,192,21037,292]

In [ ]:
L(vals, sols).zip()

(#9) [(190, True),(3267, True),(83, False),(156, False),(7290, False),(161011, False),(192, False),(21037, False),(292, True)]

In [ ]:
L(vals, sols).zip().filter(lambda o: o[1])

(#3) [(190, True),(3267, True),(292, True)]

In [ ]:
def solve(txt):
    xs = txt.splitlines()
    eqs = L(xs).map(parse)
    sols = eqs.starmap(search).map(first)
    vals = eqs.map(first)
    vals_filtered = L(vals, sols).zip().filter(lambda o: o[1])
    return vals_filtered.map(first).sum()
solve(samp)

3749

In [ ]:
solve(inp)

21572148763543

# Part 2

The engineers seem concerned; the total calibration result you gave them is nowhere close to being within safety tolerances. Just then, you spot your mistake: some well-hidden elephants are holding a third type of operator.

The concatenation operator (`||`) combines the digits from its left and right inputs into a single number. For example, 12 || 345 would become 12345. All operators are still evaluated left-to-right.

Now, apart from the three equations that could be made true using only addition and multiplication, the above example has three more equations that can be made true by inserting operators:

- `156: 15 6` can be made true through a single concatenation: `15 || 6 = 156`.
- `7290: 6 8 6 15` can be made true using `6 * 8 || 6 * 15`.
- `192: 17 8 14` can be made true using `17 || 8 + 14`.

Adding up all six test values (the three that could be made before using only + and * plus the new three that can now be made by also using ||) produces the new total calibration result of 11387.

Using your new knowledge of elephant hiding spots, determine which equations could possibly be true. What is their total calibration result?

In [ ]:
def concat(a, b):
    return int(str(a) + str(b))

Can you rewrite the concat function to use math.log10?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Sure! The idea is to figure out how many digits `b` has, then shift `a` left by that many decimal places and add `b`.

Here's how you could do it:

```python
import math

def concat(a, b):
    digits = int(math.log10(b)) + 1 if b > 0 else 1
    return a * (10 ** digits) + b
```

The `log10(b)` gives you the power of 10, and adding 1 gives you the number of digits. Then multiply `a` by `10^digits` to shift it left, and add `b`.

Does this approach make sense? Would you like to test it with some examples?

In [ ]:
import math

def concat(a, b):
    digits = int(math.log10(b)) + 1 if b > 0 else 1
    return a * (10 ** digits) + b

In [ ]:
def search2_rec(val, nums):
    if len(nums) == 1:
        return val == nums[0]
    
    if nums[0] > val:
        return False

    fst, snd, rest = nums[0], nums[1], nums[2:]
    
    for op in [concat, mul, add]:
        if search2_rec(val, [op(fst, snd)] + rest):
            return True
    
    return False

search2_rec(156, [15, 6])

True

Can you help me optimize the search2_rec function?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

I'd be happy to help! First, let me understand what optimizations you're thinking about. 

Looking at your current implementation, I can see a few potential areas:

1. **Early termination** - You already have `if nums[0] > val: return False`, which is great
2. **Memoization** - Caching results for repeated subproblems
3. **Operator ordering** - Trying operators in a smarter order (e.g., concat and mul grow faster, so try them first if nums[0] is small)
4. **Avoiding list concatenation** - Using indices instead of creating new lists

Which of these interests you most, or do you have a different optimization in mind? Also, is the current version running too slowly on the actual input?

Let's explore option 4.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Great choice! Using indices instead of creating new lists with `[op(fst, snd)] + rest` can reduce memory allocations and improve performance.

The key idea is to pass an index parameter to track your position in the original list, and accumulate the result as you go, rather than building new lists at each step.

Here's a sketch of the structure:

```python
def search2_rec(val, nums, idx=0, acc=None):
    if acc is None: acc = nums[0]
    if idx == len(nums) - 1:
        return val == acc
    # ... rest of logic
```

Can you try implementing the rest? Think about:
- How would you recurse with the next index?
- How would you apply each operator to `acc` and `nums[idx + 1]`?

Give it a shot and let me know if you get stuck!

In [ ]:
def search2(val, nums, idx=0, acc=None):
    if acc is None: acc = nums[0]
    if idx == len(nums) - 1:
        return val == acc

    if acc > val:
        return False
    
    for op in [concat, mul, add]:
        if search2_rec(val, nums, idx + 1, op(acc, nums[idx + 1])):
            return True
    
    return False

search2_rec(156, [15, 6])

True

In [ ]:
search2(156, [15, 6]), search2(7290, [6, 8, 6, 15]), search2(192, [17, 8, 14])

(True, True, True)

In [ ]:
def solve2(txt):
    xs = txt.splitlines()
    eqs = L(xs).map(parse)
    sols = eqs.starmap(search2)
    vals = eqs.map(first)
    vals_filtered = L(vals, sols).zip().filter(lambda o: o[1])
    return vals_filtered.map(first).sum()
solve2(samp)

11387

In [ ]:
solve2(inp)

581941094529163

Can you rewrite solve2 to run in parallel? Please provide full implementation.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Sure! Here's a full parallel implementation using Python's `concurrent.futures`:

```python
from concurrent.futures import ProcessPoolExecutor

def solve2(txt):
    xs = txt.splitlines()
    eqs = L(xs).map(parse)
    
    with ProcessPoolExecutor() as executor:
        sols = list(executor.starmap(search2, eqs))
    
    vals = eqs.map(first)
    vals_filtered = L(vals, sols).zip().filter(lambda o: o[1])
    return vals_filtered.map(first).sum()
```

This uses `ProcessPoolExecutor` to run `search2` on multiple equations simultaneously across different CPU cores. The `starmap` method unpacks each equation tuple (val, nums) as arguments to `search2`.

Try running `solve2(samp)` and `solve2(inp)` to see if you get a speedup on the full input!

There is no starmap

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

You're right, my mistake! `ProcessPoolExecutor` doesn't have a `starmap` method directly. 

You can use `executor.map()` with a wrapper function, or use `executor.submit()` in a loop. Here's one approach:

```python
from concurrent.futures import ProcessPoolExecutor

def solve2(txt):
    xs = txt.splitlines()
    eqs = L(xs).map(parse)
    
    with ProcessPoolExecutor() as executor:
        sols = list(executor.map(lambda eq: search2(*eq), eqs))
    
    vals = eqs.map(first)
    vals_filtered = L(vals, sols).zip().filter(lambda o: o[1])
    return vals_filtered.map(first).sum()
```

The lambda unpacks each equation tuple for `search2`. Does this work for you?

In [ ]:
def check_equation(eq):
    return search2(*eq)

def solve2_parallel(txt):
    xs = txt.splitlines()
    eqs = L(xs).map(parse)
    
    with ProcessPoolExecutor() as executor:
        sols = list(executor.map(check_equation, eqs))
    
    vals = eqs.map(first)
    vals_filtered = L(vals, sols).zip().filter(lambda o: o[1])
    return vals_filtered.map(first).sum()
solve2_parallel(samp)

11387

In [ ]:
solve2_parallel(inp)

581941094529163

Can you use parallelization from fastcore?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Yes! `fastcore` has `parallel` which is great for this. Here's how:

```python
def solve2_parallel(txt):
    xs = txt.splitlines()
    eqs = L(xs).map(parse)
    sols = parallel(check_equation, eqs)
    vals = eqs.map(first)
    vals_filtered = L(vals, sols).zip().filter(lambda o: o[1])
    return vals_filtered.map(first).sum()
```

The `parallel` function from `fastcore` handles the parallelization for you. It's simpler and integrates nicely with `L`.

Give it a try!

In [ ]:
from fastcore.parallel import parallel

def solve2_parallel_fc(txt):
    xs = txt.splitlines()
    eqs = L(xs).map(parse)
    sols = parallel(check_equation, eqs)
    vals = eqs.map(first)
    vals_filtered = L(vals, sols).zip().filter(lambda o: o[1])
    return vals_filtered.map(first).sum()
solve2_parallel_fc(inp)

581941094529163